In [3]:
import tensorflow as tf
print("TensorFlow version: ",format(tf.__version__))
import DataSets as ds
import Layers
import numpy as np
from pathlib import Path

TensorFlow version:  2.0.0


### chargement des données

In [4]:
LoadModel = False
experiment_size = 10
train = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_%dk.bin"%experiment_size), \
                   Path("/home/romain/jupyter_mla/Databases/gender_%dk.bin"%experiment_size), 1000*experiment_size)
test = ds.DataSet(Path("/home/romain/jupyter_mla/Databases/data_test10k.bin"), \
     Path("/home/romain/jupyter_mla/Databases/gender_test10k.bin"), 10000)

nb data =  10000
nb data =  10000


### classes pour définir les couches et la structure du réseau

In [5]:
class ConvNeuralNet(tf.Module):
    def __init__(self):
        list = []
        list.append( Layers.unflat('unflat',48, 48, 1) )
        nbfilter = 3
        for i in range(4):
            for j in range(2):
                list.append(Layers.conv('block_%d_conv_%d'%(i,j), output_dim=nbfilter, filterSize=3, stride=1,dropout_rate = 0.1))
            list.append( Layers.maxpool('pool', 2) )
            nbfilter *= 2
        list.append(Layers.flat())
        list.append(Layers.fc('fc', 2))
        self.list = list

    def __call__(self, x, log_summary, training):
        for l in self.list:
            x = l(x, log_summary, training)
        return x

In [6]:
def train_one_iter(model, optimizer, image, label, log_summary):
    with tf.GradientTape() as tape:
        y = model(image,log_summary,True)
        y = tf.nn.log_softmax(y)
        diff = label * y
        loss = -tf.reduce_sum(diff)
        if log_summary:
            tf.summary.scalar('cross entropy', loss)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

### Résumé du modèle

In [7]:
print ("-----------------------------------------------------")
print ("----------------------- %dk -------------------------"%experiment_size)
print ("-----------------------------------------------------")

-----------------------------------------------------
----------------------- 10k -------------------------
-----------------------------------------------------


In [ ]:
train_summary_writer = tf.summary.create_file_writer('logs %dk'%experiment_size)
optimizer = tf.optimizers.Adam(1e-3)
simple_cnn = ConvNeuralNet()

In [ ]:
for iter in range(5000):
    tf.summary.experimental.set_step(iter)

    if iter % 500 == 0:
        with train_summary_writer.as_default():
            acc1 = train.mean_accuracy(simple_cnn) * 100
            acc2 = test.mean_accuracy(simple_cnn) * 100
            print("iter= %6d accuracy - train= %.2f%% - test= %.2f%%" % (iter, acc1, acc2))

    ima, lab = train.NextTrainingBatch()
    with train_summary_writer.as_default():
        loss = train_one_iter(simple_cnn, optimizer, ima, lab, iter % 10 == 0)

    if iter % 100 == 0:
        print("iter= %6d - loss= %f" % (iter, loss))